In [82]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import re
from nltk import word_tokenize
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from nltk import RegexpTokenizer

#Middle English Imports
from cltk.tokenizers.enm import MiddleEnglishWordTokenizer
from cltk.stem.enm import stem
from cltk.tokenizers.enm import MiddleEnglishWordTokenizer

In [2]:
line_df = pd.read_csv('../data/csv/ShakespeareCharacterLines_cleaned.csv', index_col = ['play', 'name', 'line_number'])

In [3]:
colon = slice(None)

- Coriolanus is called Martius for part of the play. Make them the same character.

In [4]:
coriolanus = line_df.loc[('coriolanus', ['MARTIUS', 'CORIOLANUS'], colon), :].copy()

In [5]:
martius_line_max = coriolanus.loc[(colon, 'MARTIUS', colon), :].index.get_level_values(2).max()
coriolanus_line_max = coriolanus.loc[(colon, 'CORIOLANUS', colon), :].index.get_level_values(2).max()

In [6]:
added_index = [('coriolanus', 'CORIOLANUS', x) for x in range(coriolanus_line_max, coriolanus_line_max + martius_line_max + 1)]
coriolanus = pd.concat([coriolanus, pd.DataFrame(columns = coriolanus.columns, index = added_index)])

In [7]:
coriolanus_new_lines = coriolanus.loc[(colon, 'CORIOLANUS', colon), :].shift(martius_line_max)
coriolanus_new_lines.dropna(inplace = True)

In [8]:
martius_lines = coriolanus.loc[(colon, 'MARTIUS', colon), :].copy()
coriolanus = pd.concat([martius_lines, coriolanus_new_lines])

In [9]:
coriolanus.index = coriolanus.index.map(lambda i: (i[0], 'CORIOLANUS', i[2]))

In [10]:
coriolanus

line  \
play       name       line_number                                                      
coriolanus CORIOLANUS 1            Thanks.--What's the matter, you dissentious ro...   
                      2            He that will give good words to thee will flat...   
                      3            Hang 'em! They say? They'll sit by th' fire an...   
                      4            They are dissolved. Hang 'em! They said they w...   
                      5            Five tribunes to defend their vulgar wisdoms, ...   
...                                                                              ...   
                      183                                        Hear'st thou, Mars?   
                      184                                                        Ha?   
                      185          Measureless liar, thou hast made my heart Too ...   
                      186          Cut me to pieces, Volsces. Men and lads, Stain...   
                      187           O, that I had him, With six Aufidiuses, or mo...   

                                  play_line_number line_length  
play       name       line_number                               
coriolanus CORIOLANUS 1                         48         119  
                      2                         50        1008  
                      3                         52         477  
                      4                         54         492  
                      5                         56         293  
...                                            ...         ...  
                      183                     1086          19  
                      184                     1088           3  
                      185                     1090         363  
                      186                     1092         231  
                      187                     1097          84  

[188 rows x 3 columns]

In [11]:
line_df.drop(line_df.loc[('coriolanus', ['MARTIUS', 'CORIOLANUS'], colon), :].index, inplace = True)

In [12]:
line_df = pd.concat([line_df, coriolanus])

In [13]:
line_df.sort_index(inplace = True)

- Identify all characters that died by consulting the main texts and outside summaries for verification.

In [14]:
deaths = {'antony-and-cleopatra': ['ENOBARBUS', 'EROS', 'ANTONY', 'IRAS', 'CHARMIAN', 'CLEOPATRA'],
          'coriolanus': ['CORIOLANUS'],
          'cymbeline': ['QUEEN', 'CLOTEN'],
          'hamlet': ['POLONIUS', 'ROSENCRANTZ', 'GUILDENSTERN', 'KING', 'QUEEN', 'LAERTES', 'OPHELIA', 'HAMLET', 'OSRIC'],
          'henry-iv-part-1': ['HOTSPUR', 'VERNON', 'WORCESTER', 'BLUNT'],
          'henry-iv-part-2': ['KING'],
          'henry-v': ['HOSTESS'],
          'henry-vi-part-1': ['MORTIMER', 'BEDFORD', 'TALBOT', 'JOHN TALBOT', 'GARGRAVE', 'SALISBURY'],
          'henry-vi-part-2': ['CARDINAL', 'GLOUCESTER', 'SUFFOLK', 'CADE', 'SOMERSET', 'CLIFFORD', 'STAFFORD', 'BROTHER', 'SAYE', 'CLERK', 'HORNER', 'SOLDIER'],
          'henry-vi-part-3': ['CLIFFORD', 'FATHER', 'SON', 'WARWICK', 'MONTAGUE', 'KING HENRY', 'RUTLAND', 'YORK', 'PRINCE EDWARD'],
          'henry-viii': ['WOLSEY', 'BUCKINGHAM'],
          'julius-caesar': ['TITINIUS', 'CASSIUS', 'CAESAR', 'CINNA', 'BRUTUS', 'CATO'],
          'king-john': ['AUSTRIA', 'ARTHUR', 'KING JOHN', 'CONSTANCE', 'QUEEN ELEANOR'],
          'king-lear': ['FIRST SERVANT', 'OSWALD', 'CORDELIA', 'REGAN', 'EDMUND', 'GLOUCESTER', 'LEAR', 'CORNWALL', 'GONERIL'],
          'loves-labors-lost': ['KING'],
          'macbeth': ['DUNCAN', 'BANQUO', 'LADY MACBETH', 'YOUNG SIWARD', 'MACBETH', 'LADY MACDUFF', 'SON'],
          'othello': ['RODERIGO', 'EMILIA', 'OTHELLO', 'DESDEMONA'], 
          'pericles': ['CLEON', 'DIONYZA', 'ANTIOCHUS', 'DAUGHTER'],
          'richard-ii': ['GAUNT', 'BUSHY', 'GREEN', 'RICHARD', 'GLOUCESTER'],
          'richard-iii': ['CLARENCE', 'RICHARD', 'KING EDWARD', 'PRINCE', 'YORK', 'RIVERS', 'GREY', 'VAUGHAN', 'BRAKENBURY', 'BUCKINGHAM', 'ANNE', 'HASTINGS'],
          'romeo-and-juliet': ['ROMEO', 'JULIET', 'MERCUTIO', 'TYBALT', 'PARIS', 'LADY MONTAGUE'],
          'the-two-noble-kinsmen': ['ARCITE'],
          'the-winters-tale': ['MAMILLIUS', 'HERMIONE', 'ANTIGONUS'], 
          'timon-of-athens': ['TIMON'], 
          'titus-andronicus': ['TITUS', 'MARTIUS', 'QUINTUS', 'MUTIUS', 'TAMORA', 'SATURNINUS', 'AARON', 'BASSIANUS', 'NURSE', 'LAVINIA', 'CHIRON', 'DEMETRIUS'],
          'troilus-and-cressida': ['HECTOR', 'PATROCLUS']}

- GLOUCESTER was originally DUCHESS in richard-ii, where there are two characters with line name DUCHESS: the duchess of York and the duchess of Gloucester. She has been renamed in the text itself.

In [15]:
line_df['character_dies'] = 0

for play, name_arr in deaths.items():
    for name in name_arr:
        line_df.loc[(play, name, colon), 'character_dies'] = 1

- Get number of words per line.

In [16]:
line_df['word_count'] = line_df['line']

In [17]:
line_df['word_count'] = line_df['word_count'].map(lambda line: len(re.findall(r'(\S+)(?=(\s*|\s*\Z))', line)))

- Add new features based on total character, word, and line count.

In [18]:
line_df['total_character_count'] = 0
line_df['max_word_count'] = 0
line_df['max_line_count'] = 0

In [19]:
for index in line_df.index:
    play  = index[0]
    name = index[1]
    
    character_slice = line_df.loc[(play, name, colon)]
    
    line_df.loc[(play, name, colon), 'max_line_count'] = max(list(character_slice.index.get_level_values(2)))
    line_df.loc[(play, name, colon), 'max_word_count'] = max(list(character_slice['word_count']))
    line_df.loc[(play, name, colon), 'total_character_count'] = sum(list(character_slice['line_length']))

- Add percent total character, word, and line count features.

In [20]:
for index in line_df.index:
    play  = index[0]
    name = index[1]
    
    character_slice = line_df.loc[(play, name, colon)]
    play_slice = line_df.loc[(play, colon, colon)]
    
    line_df.loc[(play, name, colon), 'percent_line_count'] = sum(list(character_slice.index.get_level_values(2))) / sum(list(play_slice['word_count']))
    line_df.loc[(play, name, colon), 'percent_word_count'] = sum(list(character_slice['word_count'])) / sum(list(play_slice['word_count']))

- Stem and lemmatize corpus.

In [41]:
mewt = MiddleEnglishWordTokenizer()

In [85]:
#def stemmatize(sentence):
sentence = line_df.loc[('hamlet', 'HAMLET', 131)]['line'].values[0]
sentence.strip('\\')
sentence = sentence.lower()

new_sentence = ''
lemmatizer = WordNetLemmatizer()
tokenizer = RegexpTokenizer('\w+')

for word in tokenizer.tokenize(sentence):
    synonyms = wn.synsets(word)
    
    if len(synonyms) > 0:
        new_word = synonyms[0].lemma_names()[0]
    else:
        new_word = stem(word)
    
    new_word = lemmatizer.lemmatize(word)
          
    if re.search(r'\w+', word) != None:
        new_sentence += new_word + ' '

In [86]:
new_sentence

'ay so good bye to you now i am alone o what a rogue and peasant slave am i is it not monstrous that this player here but in a fiction in a dream of passion could force his soul so to his own conceit that from her working all his visage wanned tear in his eye distraction in his aspect a broken voice and his whole function suiting with form to his conceit and all for nothing for hecuba what s hecuba to him or he to hecuba that he should weep for her what would he do had he the motive and the cue for passion that i have he would drown the stage with tear and cleave the general ear with horrid speech make mad the guilty and appall the free confound the ignorant and amaze indeed the very faculty of eye and ear yet i a dull and muddy mettled rascal peak like john a dream unpregnant of my cause and can say nothing no not for a king upon whose property and most dear life a damned defeat wa made am i a coward who call me villain break my pate across pluck off my beard and blow it in my face tw